In [1]:
import numpy as np

# 讀取文件並建立每個文章的tf-idf unit vector的list
all_tf_idf_unit_vector = []
for i in range(0, 1095):
    path = f"./tf_idf_unit_vector/{i + 1}.txt"
    with open(path, 'r') as file:
        content = file.read().split()[2:]  # 跳過前兩個元素
        tf_idf_unit_vector = {int(content[j]): float(content[j + 1]) for j in range(0, len(content), 2)} # 讀取tf-idf unit vector
    all_tf_idf_unit_vector.append(tf_idf_unit_vector)

    
def getDoc(filepath):
    t_index_list = []
    tfidf_list = []
    
    # 讀取檔案並將其轉為Dataframe形式以便之後處理資料
    with open(filepath, 'r', encoding='utf-8') as file:
        # 跳過Header
        next(file)
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                t_index, tfidf = int(parts[0]), float(parts[1])
                t_index_list.append(t_index)
                tfidf_list.append(tfidf)

    data = {'t_index': t_index_list, 'tf-idf': tfidf_list}
    df = pd.DataFrame(data)
    return df
    
# 計算其unit vector
def norm(tfidf_List):
    norm_vector = sum(tfidf ** 2 for tfidf in tfidf_List) ** 0.5
    return norm_vector

def cosine(doc1, doc2):
    
    #讀取兩個檔案
    doc1_df = getDoc(doc1)
    doc2_df = getDoc(doc2)
    
    #將tf-idf欄位分別重新命名，以便merge後做內積
    doc1_df.rename(columns={'tf-idf':'docX'}, inplace=True)
    doc2_df.rename(columns={'tf-idf':'docY'}, inplace=True)  
    
    # 以t_inde做outer merge
    merged_df = pd.merge(doc1_df, doc2_df, on='t_index', how='outer').fillna(0)
    
    # 轉換為unit vector
    docX_norm = norm(merged_df['docX'].to_list())
    docY_norm = norm(merged_df['docY'].to_list())
    
    similarity = 0
    
    # 計算cosine similarity
    for index, row in merged_df.iterrows():
        similarity += (row['docX']/docX_norm) * (row['docY']/docY_norm)
    
    return similarity

# 計算cosine similarity的函式
def cosine(doc_x, doc_y):
    # 取得文章中t_index的key值
    keys_x = list(doc_x.keys())
    keys_y = list(doc_y.keys())
    union_keys = list(set(keys_x + keys_y))  # union兩文章的t_index的key值
    
    # 補零至相同維度
    vector_x = np.array([doc_x.get(key, 0.00) for key in union_keys])
    vector_y = np.array([doc_y.get(key, 0.00) for key in union_keys])
    
    # 因為已經是unit vector了，所以直接內積就好
    similarity = np.dot(vector_x, vector_y)
    
    return similarity

# 初始化similarity matrix C
n = len(all_tf_idf_unit_vector)
C = np.zeros((n, n))

for i in range(n):
    for j in range(i, n):  # 僅計算上三角部分，對角線和下三角部分對稱，提高運算效率
        sim = cosine(all_tf_idf_unit_vector[i], all_tf_idf_unit_vector[j])
        C[i, j] = sim
        C[j, i] = sim  # 對稱矩陣

# 更新cluster層次的函式
def update_cluster_hierarchy(cluster, idx1, idx2):
    cluster[idx1] += cluster[idx2]  # 合併cluster
    cluster[idx2] = []  # 清空被合併的cluster
    
# 更新similarity matrix的函式
def update_similarity_matrix(similarity_matrix, idx1, idx2):
    min_vals = np.minimum(similarity_matrix[idx1, :], similarity_matrix[idx2, :]) # complete-link，找similarity最小的merge pair
    similarity_matrix[idx1, :] = min_vals
    similarity_matrix[:, idx1] = min_vals
    similarity_matrix[idx1, idx1] = -np.inf  # 將自己與自己的similarity設為負無窮大，避免後續合併時對自己的影響
    similarity_matrix[idx2, :] = -np.inf # 清空被合併cluster的similarity
    similarity_matrix[:, idx2] = -np.inf # 清空被合併cluster的similarity
    
# 找到最大similarity位置的函式
def find_max_position(test_C):
    max_position = np.unravel_index(np.argmax(test_C), test_C.shape)  # 取得最大值所在的行和列index
    return max_position

# 將分群結果寫入文件的函式
def write_to_file(cluster, k):
    file_name = f"./{k}.txt"
    with open(file_name, "w") as f:
        for group in cluster:
            if group:
                # 將分群結果寫入文件
                f.write('\n')
                f.write('\n'.join(map(str, sorted(group))))  # 將文章id排序後寫入文件
                f.write('\n')

# 分成8、13、20群，並將各自結果輸出
for k in [8, 13, 20]:
    output_k = k
    test_C = np.copy(C)
    np.fill_diagonal(test_C, 0)  # 將similarity matrix的對角線值設為零

    clusters = [[i + 1] for i in range(len(test_C))]  # 初始化每個文章為各自為一個獨立的cluster

    # apply simple HAC，直到達到指定的cluster數量
    while k < len(test_C):
        max_pos = max(find_max_position(test_C))  # 找到similarity matrix中similarity最大的位置
        min_pos = min(find_max_position(test_C))  # 找到similarity matrix中similarity最小的位置
        update_cluster_hierarchy(clusters, min_pos, max_pos)  # 更新cluster層次
        update_similarity_matrix(test_C, min_pos, max_pos)  # 更新similarity matrix
        k += 1

    clusters = [group for group in clusters if group]  # 去除空的cluster

    for group in clusters:
        group.sort()  # 對每個群組的文章id進行排序

    # 將分群結果寫入文件
    write_to_file(clusters, output_k)

In [2]:
clusters

[[1,
  2,
  14,
  36,
  58,
  69,
  80,
  91,
  102,
  110,
  111,
  112,
  114,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  125,
  126,
  127,
  128,
  129,
  131,
  142,
  153,
  164,
  175,
  186,
  190,
  208,
  209,
  220,
  231,
  238,
  242,
  245,
  253,
  264,
  274,
  275,
  286,
  297,
  319,
  320,
  342,
  353,
  364,
  375,
  386,
  397,
  408,
  419,
  430,
  431,
  442,
  464,
  486,
  497,
  508,
  519,
  530,
  541,
  542,
  553,
  564,
  575,
  586,
  597,
  608,
  619,
  630,
  641,
  652,
  653,
  664,
  675,
  686,
  688,
  689,
  697,
  707,
  708,
  719,
  724,
  750,
  751,
  752,
  763,
  775,
  778,
  786,
  796,
  797,
  808,
  819,
  830,
  841,
  852,
  863,
  874,
  875,
  886,
  897,
  919,
  930,
  952,
  963,
  974,
  985,
  986,
  1019,
  1030,
  1041,
  1052,
  1063,
  1074,
  1085],
 [3,
  25,
  326,
  330,
  331,
  337,
  343,
  349,
  350,
  351,
  355,
  358,
  365,
  369,
  370,
  377,
  381,
  384,
  387,
  389,
  392,
  398,
  39

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.DataFrame(C)
df

,0,1,2,3,4,5,6,7,8,9,...,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094
0,1.000000,0.274062,0.070954,0.005275,0.003546,0.000745,0.026321,0.018369,0.034803,0.029372,...,0.018785,0.012769,0.020962,0.023818,0.019857,0.012990,0.004447,0.005462,0.008508,0.008732
1,0.274062,1.000000,0.090476,0.001033,0.000973,0.001231,0.006087,0.007773,0.015990,0.011029,...,0.005040,0.002687,0.002342,0.008655,0.006356,0.003468,0.004393,0.004090,0.002014,0.005174
2,0.070954,0.090476,1.000000,0.013685,0.012655,0.001488,0.047869,0.012113,0.037552,0.006820,...,0.016784,0.007838,0.016968,0.034788,0.017758,0.014633,0.004617,0.004716,0.028689,0.033462
3,0.005275,0.001033,0.013685,1.000000,0.000000,0.000000,0.004444,0.000193,0.016828,0.004314,...,0.042864,0.007156,0.392724,0.008232,0.435561,0.016790,0.000000,0.000238,0.018693,0.000672
4,0.003546,0.000973,0.012655,0.000000,1.000000,0.014806,0.003673,0.395287,0.005603,0.018634,...,0.003519,0.078663,0.015141,0.000000,0.013163,0.000639,0.026737,0.028059,0.000467,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,0.012990,0.003468,0.014633,0.016790,0.000639,0.000809,0.319474,0.008748,0.366811,0.178126,...,0.011407,0.000513,0.011637,0.018196,0.008682,1.000000,0.001600,0.001652,0.753042,0.278006
1091,0.004447,0.004393,0.004617,0.000000,0.026737,0.534307,0.019184,0.064002,0.004896,0.003602,...,0.013553,0.195392,0.003580,0.002696,0.007244,0.001600,1.000000,0.961097,0.002354,0.000000
1092,0.005462,0.004090,0.004716,0.000238,0.028059,0.497415,0.019025,0.069582,0.006006,0.003353,...,0.012617,0.201956,0.011665,0.003041,0.008511,0.001652,0.961097,1.000000,0.002783,0.000000
1093,0.008508,0.002014,0.028689,0.018693,0.000467,0.000591,0.346288,0.009252,0.436835,0.221451,...,0.012638,0.010913,0.015476,0.020948,0.009967,0.753042,0.002354,0.002783,1.000000,0.354109
